# Recording

> Storing data

In [ ]:
#|default_exp cb_groups

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *
from isaacai.models import *
from isaacai.initialization import *
from isaacai.trainer import *
from isaacai.training import *
from isaacai.recording import *
from isaacai.visualization import *

from datetime import datetime, timedelta
import torchvision.transforms.functional as TF,torch.nn.functional as F
import math, time

import matplotlib.pyplot as plt
import matplotlib as mpl
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd , numpy as np
from torcheval.metrics import MulticlassAccuracy,Mean
from torch.optim.lr_scheduler import ExponentialLR

import dill as pickle
from fastprogress.fastprogress import master_bar, progress_bar
import inspect
import torchinfo


In [ ]:
#|hide
import logging

In [ ]:
#|hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
# mpl.rcParams['image.cmap'] = 'gray'

logging.disable(logging.WARNING)

set_seed(42)


In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dls = DataLoaders.from_dataset_dict(_dataset, 256, num_workers=4)

In [ ]:
#| export
class CoreCBs:
    def __init__(self,device=def_device,module_filter=fc.noop,**metrics):
        self.callbacks = [DeviceCB(device=device),
                          BasicTrainCB(),
                          MetricsCB(**metrics),
                          ProgressCB(),
                          ActivationStatsCB(module_filter)]
        


In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  get_model_conv(),
                  callbacks=[CoreCBs(Accuracy=MulticlassAccuracy()),NBatchCB(3)])

In [ ]:
trainer.fit()

,train,valid,Accuracy
0,0.845056,0.557575,0.794


,train,valid,Accuracy
1,0.483043,0.474307,0.828400


,train,valid,Accuracy
2,0.421664,0.432006,0.842200


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()